### Data loader

In [2]:
import sys
import open3d as o3d
import torch
import pickle as pkl
import torch.nn as nn
import argparse
from PIL import Image
import os
import numpy as np
import pcl

#dat_dir = '/home/kulunu/Desktop/ADAPT_Competition/ADAPT_Pose_Estimation/ADAPT_FB2/validation/'
#dat_dir = '/home/kulunu/PVN3D/pvn3d/datasets/Adapt/Adapt_dataset'
dat_dir = '/home/kulunu/Desktop/ADAPT_Competition/ADAPT_Pose_Estimation_test_split/adapt_fb2_test/test/'
mesh_files_path = '/home/kulunu/PVN3D/pvn3d/datasets/Adapt/Adapt_dataset/models'
corners_txt_path = '/home/kulunu/PVN3D/pvn3d/datasets/Adapt/Adapt_object_kps/'
n_models = 3

scene_id_leading_zeros = 6
img_id_leading_zeros = 6
rgb_format = '.jpg'
dpt_format = '.png'
cam_scale = 1
use_given_K = True


class Adapt(object):
    
    def __init__(self, scene_id):

        self.dir = dat_dir + str(scene_id).zfill(scene_id_leading_zeros)
        #self.dir = dat_dir
        self.rgb_dir = self.dir +'/rgb'  
        self.dpt_dir = self.dir+'/depth'    
        self.mesh = {}
        self.corners = {}

        for i in range(1, n_models + 1):
                self.mesh[str(i)] = np.array(pcl.load(mesh_files_path+'/obj_'+str(i)+'.pcd'))
                self.corners[str(i)] = np.loadtxt(corners_txt_path+str(i)+'/corners.txt')

    def __getitem__(self, index):
        

        with Image.open(os.path.join(self.rgb_dir,str(im_id).zfill(img_id_leading_zeros)+ rgb_format)) as ri:
            rgb = np.array(ri)[:, :, :3]
        rgb = np.transpose(rgb, (2, 0, 1))
        with Image.open(os.path.join(self.dpt_dir,str(im_id).zfill(img_id_leading_zeros)+ dpt_format)) as di:
            dpt = np.array(di)
            dpt = dpt/65535         
            dpt = dpt*30.0  
        if use_given_K :
                    with open(os.path.join(self.dir,'scene_camera.json'), "r") as f:
                        mat = json.load(f)
                        K = np.array(mat[str(im_id)]['cam_K']).reshape(3,3)
        else :
             K = np.array([[615.1397026909722, 0.0, 322.78683465719223], [0.0, 615.4323641853988, 244.22407517651774], [0.0, 0.0, 1.0]]) #camera insintric

        cld, choose = bs_utils.dpt_2_cld(dpt, cam_scale, K)
        rgb_lst = []

        for ic in range(rgb.shape[0]):
            rgb_lst.append(
                rgb[ic].flatten()[choose].astype(np.float32)
            )
        rgb_pt = np.transpose(np.array(rgb_lst), (1, 0)).copy()
        choose = np.array([choose])
        choose_2 = np.array([i for i in range(len(choose[0, :]))])

        if len(choose_2) < 400:
            print("Valid points are less than 400")
        if len(choose_2) > config.n_sample_points:
            c_mask = np.zeros(len(choose_2), dtype=int)
            c_mask[:config.n_sample_points] = 1
            np.random.shuffle(c_mask)
            choose_2 = choose_2[c_mask.nonzero()]
            print("Valid points are more than configured")
        else:
            leng = len(choose_2)
            choose_2 = np.pad(choose_2, (0, config.n_sample_points-len(choose_2)), 'wrap')
            print("Valid points are optimum")

        cld_rgb = np.concatenate((cld, rgb_pt), axis=1)
        cld_rgb = cld_rgb[choose_2, :]
        cld = cld[choose_2, :]
        normal = bs_utils.get_normal(cld)[:, :3]
        normal[np.isnan(normal)] = 0.0
        cld_rgb_nrm = np.concatenate((cld_rgb, normal), axis=1)
        choose = choose[:, choose_2]
        self.data_torch = [torch.from_numpy(rgb[np.newaxis, ...].astype(np.float32)), torch.from_numpy(cld_rgb_nrm[np.newaxis, ...].astype(np.float32)),torch.LongTensor(choose[np.newaxis, ...].astype(np.int32))]

        return data_torch


### Data_torch ==> Model